# Data cleaning

In [85]:
#import libraries

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import requests
import json
import pandas as pd
from pandas.io.json import json_normalize
import os
import re
import random

## French museums visitors

In [131]:
#import file about the french museums visitors

visitors=pd.read_csv('frequentation-des-musees-de-france.csv', sep=';', encoding='UTF8')
visitors.head()

,REF DU MUSEE,NOM DU MUSEE,Année,REGION,VILLE,DATE APPELLATION,PAYANT,GRATUIT,TOTAL,Note,ID MUSEOFILE,NOMDEP,Observations
0,8109903,Musée de l'Abbaye Saint-Michel,2009,MIDI-PYRENEES,GAILLAC,01/02/2003,2613.0,669.0,3282.0,NaN,NaN,TARN,NaN
1,8114501,Musée Raymond Lafage,2009,MIDI-PYRENEES,LISLE-SUR-TARN,01/02/2003,1898.0,4429.0,6327.0,NaN,NaN,TARN,NaN
2,5904301,Musée Benoît de Puydt,2009,NORD-PAS-DE-CALAIS,BAILLEUL,01/02/2003,6590.0,0.0,6590.0,NaN,NaN,NORD,NaN
3,5912202,Musée Diocésain d'Art Sacré,2009,NORD-PAS-DE-CALAIS,CAMBRAI,01/02/2003,NaN,NaN,NaN,F,NaN,NORD,NaN
4,5918303,Musée Portuaire,2009,NORD-PAS-DE-CALAIS,DUNKERQUE,01/02/2003,24543.0,21642.0,46185.0,NaN,NaN,NORD,NaN


### Investigating data

In [132]:
#check the data types

visitors.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23586 entries, 0 to 23585
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   REF DU MUSEE      23586 non-null  object 
 1   NOM DU MUSEE      23586 non-null  object 
 2   Année             23586 non-null  int64  
 3   REGION            23586 non-null  object 
 4   VILLE             23586 non-null  object 
 5   DATE APPELLATION  21578 non-null  object 
 6   PAYANT            20235 non-null  float64
 7   GRATUIT           20219 non-null  float64
 8   TOTAL             20218 non-null  float64
 9   Note              3109 non-null   object 
 10  ID MUSEOFILE      2273 non-null   object 
 11  NOMDEP            23586 non-null  object 
 12  Observations      651 non-null    object 
dtypes: float64(3), int64(1), object(9)
memory usage: 2.3+ MB


We can change the data types of PAYANT, GRATUIT, TOTAL since those are floats and can be converted into integers since they are related to the number of visitors in each museum. But first, we should check if there are values equal to 0 or missing values in order to avoid changing the meaning when doing the conversion.

### Handling null and missing values

In [133]:
#check the missing values in either one of the columns: PAYANT / GRATUIT / TOTAL

visitors.loc[(visitors['GRATUIT'].isna()) | (visitors['PAYANT'].isna()) | (visitors['TOTAL'].isna())]

,REF DU MUSEE,NOM DU MUSEE,Année,REGION,VILLE,DATE APPELLATION,PAYANT,GRATUIT,TOTAL,Note,ID MUSEOFILE,NOMDEP,Observations
3,5912202,Musée Diocésain d'Art Sacré,2009,NORD-PAS-DE-CALAIS,CAMBRAI,01/02/2003,NaN,NaN,NaN,F,NaN,NORD,NaN
36,5306201,Musée d'Art et d'Archéologie Hôtel Fouquet,2009,PAYS DE LA LOIRE,CHATEAU-GONTIER,01/02/2003,NaN,NaN,NaN,F,NaN,MAYENNE,NaN
37,7207101,Musée Heurteloup-Chevalier,2009,PAYS DE LA LOIRE,CHATEAU-DU-LOIR,01/02/2003,NaN,NaN,NaN,F,NaN,SARTHE,NaN
42,268401,Musée de la Vie Rurale et Forestière,2009,PICARDIE,SAINT-MICHEL-EN-THIERACHE,01/02/2003,NaN,NaN,NaN,F,NaN,AISNE,NaN
58,7921601,Musée des Ruralies- Conservatoire du Machinism...,2009,POITOU-CHARENTES,PRAHECQ,01/02/2003,NaN,NaN,NaN,F,NaN,DEUX-SEVRES,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
23563,7207101,Musée Heurteloup-Chevalier,2020,PAYS DE LA LOIRE,CHATEAU-DU-LOIR,01/02/2003,NaN,NaN,NaN,F,M0769,SARTHE,NaN
23566,416601,Musée Archéologique,2020,PROVENCE-ALPES-CÔTE D'AZUR,RIEZ-LA-ROMAINE,01/02/2003,NaN,NaN,NaN,F,M0862,ALPES DE HAUTE PROVENCE,NaN
23570,608809,Musée du Vieux-Logis de Nice,2020,PROVENCE-ALPES-CÔTE D'AZUR,NICE,01/02/2003,NaN,NaN,NaN,F,M0882,ALPES-MARITIMES,NaN
23573,1300106,Musée Paul Arbaud,2020,PROVENCE-ALPES-CÔTE D'AZUR,AIX-EN-PROVENCE,01/02/2003,NaN,NaN,NaN,F,M0899,BOUCHES DU RHONE,NaN


In [114]:
#check the values equal to 0 in all the 3 columns: PAYANT / GRATUIT / TOTAL

visitors.loc[(visitors['GRATUIT'] == 0) & (visitors['PAYANT'] == 0) & (visitors['TOTAL'] == 0)]

,REF DU MUSEE,NOM DU MUSEE,Année,REGION,VILLE,DATE APPELLATION,PAYANT,GRATUIT,TOTAL,Note,ID MUSEOFILE,NOMDEP,Observations
4430,3726104,Musée dU Vin de Touraine,2019,CENTRE-VAL DE LOIRE,TOURS,01/02/2003,0.0,0.0,0.0,NaN,M0267,INDRE ET LOIRE,NaN
8926,3808201,Musée - Parc Archéologique du Lac de Paladru,2019,AUVERGNE-RHONE-ALPES,CHARAVINES,01/02/2003,0.0,0.0,0.0,NaN,M1187,ISERE,NaN
9010,238101,Musée - Centre de Documentation Alfred Desmasures,2019,HAUTS-DE-France,HIRSON,01/02/2003,0.0,0.0,0.0,NaN,M0788,AISNE,NaN
13524,3815101,Musée Géo-Charles,2019,AUVERGNE-RHONE-ALPES,ECHIROLLES,01/02/2003,0.0,0.0,0.0,NaN,M0992,ISERE,NaN
16354,1038701,Muséum d'Histoire Naturelle (Musée Pic Saint-L...,2002,CHAMPAGNE-ARDENNES,TROYES,NaN,0.0,0.0,0.0,NaN,NaN,AUBE,1038701 ; 1038704
17196,1038702,Musée Vauluisant - Musée Historique de Troyes ...,2002,CHAMPAGNE-ARDENNES,TROYES,NaN,0.0,0.0,0.0,NaN,NaN,AUBE,1038702 ; 1038703
23455,9109701,Musée Dunoyer de Segonzac,2020,ILE-DE-France,BOUSSY-SAINT-ANTOINE,01/02/2003,0.0,0.0,0.0,NaN,M0402,ESSONNE,NaN


In [134]:
#since the combination of the missing values in either one of the columns and the values equal to 0 in all the 3 columns represent 3387 rows over 23586 rows, I will drop these rows

visitors.drop(visitors.loc[(visitors['GRATUIT'].isna()) | (visitors['PAYANT'].isna()) | (visitors['TOTAL'].isna())].index, inplace=True)

### Change the data types

In [135]:
#change the data types from float to integers

visitors['PAYANT']=visitors['PAYANT'].astype(int)
visitors['GRATUIT']=visitors['GRATUIT'].astype(int)
visitors['TOTAL']=visitors['TOTAL'].astype(int)

In [121]:
visitors.describe()

,Année,PAYANT,GRATUIT,TOTAL
count,20206.000000,2.020600e+04,2.020600e+04,2.020600e+04
mean,2010.533406,3.226519e+04,2.116577e+04,5.343096e+04
std,5.703691,2.454964e+05,1.168697e+05,3.515464e+05
min,2001.000000,0.000000e+00,0.000000e+00,0.000000e+00
25%,2006.000000,8.722500e+02,1.157000e+03,4.118250e+03
50%,2011.000000,3.996000e+03,4.421000e+03,1.105650e+04
75%,2015.000000,1.421725e+04,1.372175e+04,2.980475e+04
max,2020.000000,6.676633e+06,4.117897e+06,1.010596e+07


In [122]:
visitors.isna().sum()

REF DU MUSEE            0
NOM DU MUSEE            0
Année                   0
REGION                  0
VILLE                   0
DATE APPELLATION     1810
PAYANT                  0
GRATUIT                 0
TOTAL                   0
Note                20206
ID MUSEOFILE        18284
NOMDEP                  0
Observations        19815
dtype: int64

There is a lot of missing values regarding the columns: DATE APPELLATION, PAYANT, GRATUIT, TOTAL, Note, ID MUSEOFILE, Observations

### Dropping columns

In [136]:
#drop the columns that won't be used (and that contain too many missing values)

visitors.drop(columns=['DATE APPELLATION', 'Note', 'Observations'], inplace=True)

## French museums infos

In [138]:
#import file about the french museums infos

museums=pd.read_csv('liste_des_musees_franciliens.csv', sep=';', encoding='UTF8')
museums.head()

,Région administrative,Département,Identifiant Muséofile,Commune,Nom officiel du musée,Adresse,Lieu,Code Postal,Téléphone,URL,Latitude,Longitude,REF_Deps,geolocalisation,Date arrêté attribution appellation
0,Île-de-France,Hauts-de-Seine,M0411,Issy-les-Moulineaux,musée français de la carte à jouer et galerie ...,"16, rue Auguste Gervais",NaN,92130,01 41 23 83 60,www.museecarteajouer.com/,48.822771,2.273453,9204001.0,"48.822771, 2.273453",01/02/2003
1,Île-de-France,Hauts-de-Seine,M0413,Neuilly-sur-Seine,musée des automates,12 rue du centre,NaN,92200,01 47 45 29 40,www.neuillysurseine.fr/details-d-un-acteur?id=240,48.879608,2.252754,9205101.0,"48.879608, 2.252754",01/02/2003
2,Île-de-France,Hauts-de-Seine,M5019,Sèvres,musée national de la céramique,Place de la Manufacture,NaN,92310,01 46 29 38 01,NaN,48.817037,2.205733,9207201.0,"48.817037, 2.205733",06/01/2002
3,Île-de-France,Paris,M1114,Paris,maison de Victor Hugo,6 place des Vosges,Hôtel de Rohan-Guéménée,75004,01 42 72 10 16,www.maisonsvictorhugo.paris.fr/,48.854808,2.366232,7510405.0,"48.854808, 2.366232",08/07/2004
4,Île-de-France,Paris,M1104,Paris,musée Carnavalet - histoire de Paris,"23, rue de Sévigné",Hôtel Carnavalet,75003,01 44 59 58 58,www.carnavalet.paris.fr/,48.856990,2.362856,7510307.0,"48.85699, 2.362856",08/07/2004


### Investigating data

In [26]:
#check the data types

museums.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 133 entries, 0 to 132
Data columns (total 15 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   Région administrative                133 non-null    object 
 1   Département                          133 non-null    object 
 2   Identifiant Muséofile                133 non-null    object 
 3   Commune                              133 non-null    object 
 4   Nom officiel du musée                133 non-null    object 
 5   Adresse                              125 non-null    object 
 6   Lieu                                 63 non-null     object 
 7   Code Postal                          133 non-null    int64  
 8   Téléphone                            126 non-null    object 
 9   URL                                  126 non-null    object 
 10  Latitude                             133 non-null    float64
 11  Longitude                       

In [27]:
museums.describe()

,Code Postal,Latitude,Longitude,REF_Deps
count,133.000000,133.000000,133.000000,1.300000e+02
mean,82029.932331,48.473681,2.697111,8.171023e+06
std,8239.034049,4.023792,4.051750,7.981958e+05
min,75001.000000,2.449719,1.719901,7.510102e+06
25%,75015.000000,48.808966,2.234894,7.510903e+06
50%,77580.000000,48.858133,2.323672,7.733302e+06
75%,92150.000000,48.878849,2.393807,9.202027e+06
max,95500.000000,49.112285,48.986516,9.550002e+06


In [28]:
museums.isna().sum()

Région administrative                   0
Département                             0
Identifiant Muséofile                   0
Commune                                 0
Nom officiel du musée                   0
Adresse                                 8
Lieu                                   70
Code Postal                             0
Téléphone                               7
URL                                     7
Latitude                                0
Longitude                               0
REF_Deps                                3
geolocalisation                         0
Date arrêté attribution appellation     0
dtype: int64

### Dropping columns

In [140]:
#drop the columns that won't be used

museums.drop(columns=['Date arrêté attribution appellation'], inplace=True)

## French exhibitions

In [29]:
#import file about the french museums infos

#pd.set_option('display.max_columns', None)

exhibitions=pd.read_csv('que-faire-a-paris.csv', sep=';', encoding='UTF8')
exhibitions.head()



,ID,URL,Titre,Chapeau,Description,Date de début,Date de fin,Occurrences,Description de la date,URL de l'image,...,Unnamed: 189,Unnamed: 190,Unnamed: 191,Unnamed: 192,Unnamed: 193,Unnamed: 194,Unnamed: 195,Unnamed: 196,Unnamed: 197,Unnamed: 198
0,34155,https://www.paris.fr/evenements/baila-babibou-...,"Baila Babibou : spectacle danse & musique, dès...",Baila Babibou propose un dialogue entre une mé...,<p>La danseuse démarre le spectacle en chrysal...,2023-06-21T13:30:00+02:00,2023-06-21T14:00:00+02:00,2023-06-21T11:30:00+02:00_2023-06-21T12:00:00+...,Le mercredi 21 juin 2023<br />de 10h30 à 11h00...,https://cdn.paris.fr/qfapv4/2023/03/17/huge-06...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,34268,https://www.paris.fr/evenements/le-fil-des-noe...,Le fil des nœuds,Exposition collective de La.ima du 10 juin au ...,<p>Qu'y a-t-il au-delà de la peau qui cache la...,2023-06-10T02:00:00+02:00,2023-07-09T01:59:59+02:00,NaN,Du samedi 10 juin 2023 au samedi 08 juillet 20...,https://cdn.paris.fr/qfapv4/2023/03/19/huge-04...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,34245,https://www.paris.fr/evenements/parcours-d-int...,Parcours d’intellectuels en exil : un humanism...,Un cycle de 4 conférences dédié aux trajectoir...,<p>L’exil n’a jamais épargné les intellectuels...,2023-03-28T02:00:00+02:00,2023-06-14T01:59:59+02:00,NaN,Du mardi 28 mars 2023 au mardi 13 juin 2023 : ...,https://cdn.paris.fr/qfapv4/2023/03/18/huge-e3...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,33666,https://www.paris.fr/evenements/concert-java-m...,"Concert : Java, musique du pays Sunda",Venez découvrir la musique du pays Sunda de l'...,"<p>Basé à Paris, <strong>Gamelan Degung Sora G...",2023-05-26T22:00:00+02:00,2023-05-27T00:00:00+02:00,2023-05-26T20:00:00+02:00_2023-05-26T22:00:00+...,Le vendredi 26 mai 2023<br />de 19h00 à 21h00<...,https://cdn.paris.fr/qfapv4/2023/03/09/huge-7f...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,34251,https://www.paris.fr/evenements/p-tit-dej-litt...,P'tit-Déj' Littéraire,Les bibliothécaires vous lisent leurs derniers...,"<p>Depuis quelques éditions, cette animation e...",2023-06-10T13:30:00+02:00,2023-06-10T15:00:00+02:00,2023-06-10T11:30:00+02:00_2023-06-10T13:00:00+...,Le samedi 10 juin 2023<br />de 10h30 à 12h00<b...,https://cdn.paris.fr/qfapv4/2023/03/04/huge-a3...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [30]:
#pd.set_option('display.max_rows', None)

exhibitions.isna().sum()

ID                 0
URL                0
Titre              0
Chapeau            5
Description       19
                ... 
Unnamed: 194    2077
Unnamed: 195    2076
Unnamed: 196    2076
Unnamed: 197    2077
Unnamed: 198    2076
Length: 199, dtype: int64

In [31]:
#remove "Unnamed" columns

exhibitions = exhibitions.loc[:, ~exhibitions.columns.str.contains('^Unnamed')]
exhibitions

,ID,URL,Titre,Chapeau,Description,Date de début,Date de fin,Occurrences,Description de la date,URL de l'image,...,Date de mise à jour,Image de couverture,Programmes,En ligne - address_url,En ligne - address_url_text,En ligne - address_text,title_event,audience,childrens,contributor_group
0,34155,https://www.paris.fr/evenements/baila-babibou-...,"Baila Babibou : spectacle danse & musique, dès...",Baila Babibou propose un dialogue entre une mé...,<p>La danseuse démarre le spectacle en chrysal...,2023-06-21T13:30:00+02:00,2023-06-21T14:00:00+02:00,2023-06-21T11:30:00+02:00_2023-06-21T12:00:00+...,Le mercredi 21 juin 2023<br />de 10h30 à 11h00...,https://cdn.paris.fr/qfapv4/2023/03/17/huge-06...,...,2023-03-17T17:30:43+01:00,https://opendata.paris.fr/api/explore/v2.1/cat...,NaN,NaN,NaN,NaN,NaN,Public enfants. A partir de 1 ans. Jusqu'à 7 ans.,NaN,Bibliothèques
1,34268,https://www.paris.fr/evenements/le-fil-des-noe...,Le fil des nœuds,Exposition collective de La.ima du 10 juin au ...,<p>Qu'y a-t-il au-delà de la peau qui cache la...,2023-06-10T02:00:00+02:00,2023-07-09T01:59:59+02:00,NaN,Du samedi 10 juin 2023 au samedi 08 juillet 20...,https://cdn.paris.fr/qfapv4/2023/03/19/huge-04...,...,2023-03-19T15:06:21+01:00,https://opendata.paris.fr/api/explore/v2.1/cat...,NaN,NaN,NaN,NaN,David Tanné,Public adolescents adultes.,NaN,Bibliothèques
2,34245,https://www.paris.fr/evenements/parcours-d-int...,Parcours d’intellectuels en exil : un humanism...,Un cycle de 4 conférences dédié aux trajectoir...,<p>L’exil n’a jamais épargné les intellectuels...,2023-03-28T02:00:00+02:00,2023-06-14T01:59:59+02:00,NaN,Du mardi 28 mars 2023 au mardi 13 juin 2023 : ...,https://cdn.paris.fr/qfapv4/2023/03/18/huge-e3...,...,2023-03-20T09:56:18+01:00,https://opendata.paris.fr/api/explore/v2.1/cat...,NaN,NaN,NaN,NaN,NaN,Public adolescents adultes.,Conférence : Celso Furtado : un exil académiqu...,Internautes
3,33666,https://www.paris.fr/evenements/concert-java-m...,"Concert : Java, musique du pays Sunda",Venez découvrir la musique du pays Sunda de l'...,"<p>Basé à Paris, <strong>Gamelan Degung Sora G...",2023-05-26T22:00:00+02:00,2023-05-27T00:00:00+02:00,2023-05-26T20:00:00+02:00_2023-05-26T22:00:00+...,Le vendredi 26 mai 2023<br />de 19h00 à 21h00<...,https://cdn.paris.fr/qfapv4/2023/03/09/huge-7f...,...,2023-03-22T11:02:35+01:00,https://opendata.paris.fr/api/explore/v2.1/cat...,NaN,NaN,NaN,NaN,NaN,Tout public.,NaN,Bibliothèques
4,34251,https://www.paris.fr/evenements/p-tit-dej-litt...,P'tit-Déj' Littéraire,Les bibliothécaires vous lisent leurs derniers...,"<p>Depuis quelques éditions, cette animation e...",2023-06-10T13:30:00+02:00,2023-06-10T15:00:00+02:00,2023-06-10T11:30:00+02:00_2023-06-10T13:00:00+...,Le samedi 10 juin 2023<br />de 10h30 à 12h00<b...,https://cdn.paris.fr/qfapv4/2023/03/04/huge-a3...,...,2023-03-22T13:45:16+01:00,https://opendata.paris.fr/api/explore/v2.1/cat...,NaN,NaN,NaN,NaN,NaN,Public adolescents adultes.,NaN,Bibliothèques
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2072,35588,https://www.paris.fr/evenements/projection-du-...,"Projection du film ""Icare"", accessible aux sou...",Venez profiter d'une projection gratuite du fi...,"<p>Dans le cadre du Quartier du livre 2023, le...",2023-06-04T13:45:00+02:00,2023-06-04T15:00:00+02:00,2023-06-04T11:45:00+02:00_2023-06-04T13:00:00+...,Le dimanche 04 juin 2023<br />de 10h45 à 12h00...,https://cdn.paris.fr/qfapv4/2023/04/20/huge-b7...,...,2023-04-21T10:10:45+02:00,https://opendata.paris.fr/api/explore/v2.1/cat...,NaN,NaN,NaN,NaN,Séance accessible : Icare,Tout public.,NaN,Agenda
2073,34237,https://www.paris.fr/evenements/bhlm-2023-la-l...,BHLM 2023 : La lecture prend l'air !,C’est l’été ! À partir du 13 juillet et jusqu'...,<p>Profitez d'un moment de détente estival ave...,2023-07-13T18:30:00+02:00,2023-08-10T20:00:00+02:00,2023-07-13T16:30:00+02:00_2023-07-13T18:00:00+...,Le vendredi 04 août 2023<br />de 10h00 à 12h00...,https://cdn.paris.fr/qfapv

In [32]:
exhibitions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2077 entries, 0 to 2076
Data columns (total 42 columns):
 #   Column                       Non-Null Count  Dtype 
---  ------                       --------------  ----- 
 0   ID                           2077 non-null   object
 1   URL                          2077 non-null   object
 2   Titre                        2077 non-null   object
 3   Chapeau                      2072 non-null   object
 4   Description                  2058 non-null   object
 5   Date de début                2021 non-null   object
 6   Date de fin                  2021 non-null   object
 7   Occurrences                  1799 non-null   object
 8   Description de la date       2016 non-null   object
 9   URL de l'image               2072 non-null   object
 10  Texte alternatif de l'image  1047 non-null   object
 11  Crédit de l'image            1876 non-null   object
 12  Mots clés                    2025 non-null   object
 13  Nom du lieu                  1945

In [33]:
exhibitions.describe()

,ID,URL,Titre,Chapeau,Description,Date de début,Date de fin,Occurrences,Description de la date,URL de l'image,...,Date de mise à jour,Image de couverture,Programmes,En ligne - address_url,En ligne - address_url_text,En ligne - address_text,title_event,audience,childrens,contributor_group
count,2077,2077,2077,2072,2058,2021,2021,1799,2016,2072,...,2072,2072,548,27,26,88,902,2071,79,2072
unique,2077,2077,2008,1938,2003,1091,1046,1442,1619,1977,...,2071,1977,93,25,24,83,811,173,79,14
top,34155,https://www.paris.fr/evenements/baila-babibou-...,La Jam du Lundi animée par François Constantin,"Concerts gratuits, Nuits rock et Disquaire à P...",<h3>28 crèches sont ouvertes les samedis matin...,2023-06-03T22:00:00+02:00,2023-06-04T02:00:00+02:00,2023-06-03T20:00:00+02:00_2023-06-03T02:00:00+...,Le samedi 03 juin 2023<br />de 19h00 à 01h00<b...,https://cdn.paris.fr/qfapv4/2023/01/12/huge-85...,...,2023-05-10T19:04:55+02:00,https://opendata.paris.fr/api/explore/v2.1/cat...,Nuit Blanche : le programme à Paris (https://w...,Bande annonce,Bande annonce,Plusieurs lieux dans Paris,26e Festival de l'Imaginaire,Tout public.,Conférence : Celso Furtado : un exil académiqu...,Internautes
freq,1,1,5,15,22,113,36,26,26,22,...,2,22,100,2,2,4,13,1075,1,716


In [34]:
#API - à voir

#response = requests.get('https://opendata.paris.fr/api/records/1.0/search/?dataset=que-faire-a-paris-&q=&facet=date_start&facet=date_end&facet=tags&facet=address_name&facet=address_zipcode&facet=address_city&facet=pmr&facet=blind&facet=deaf&facet=transport&facet=price_type&facet=access_type&facet=updated_at&facet=programs')
#response

## Merging french museums visitors & infos

In [35]:
df = visitors.merge(museums, left_on="ID MUSEOFILE", right_on="Identifiant Muséofile")
df.head()

,REF DU MUSEE,NOM DU MUSEE,Année,REGION,VILLE,DATE APPELLATION,PAYANT,GRATUIT,TOTAL,Note,...,Adresse,Lieu,Code Postal,Téléphone,URL,Latitude,Longitude,REF_Deps,geolocalisation,Date arrêté attribution appellation
0,9109701,Musée Dunoyer de Segonzac,2019,ILE-DE-France,BOUSSY-SAINT-ANTOINE,01/02/2003,0,172,172,NaN,...,"5, place des Droits de l'Homme",NaN,91800,01 69 00 49 23,www.ville-boussy.fr/votre-mairie/la-culture.html,48.688925,2.530019,9109701.0,"48.688925, 2.530019",01/02/2003
1,9109701,Musée Dunoyer de Segonzac,2020,ILE-DE-France,BOUSSY-SAINT-ANTOINE,01/02/2003,0,0,0,NaN,...,"5, place des Droits de l'Homme",NaN,91800,01 69 00 49 23,www.ville-boussy.fr/votre-mairie/la-culture.html,48.688925,2.530019,9109701.0,"48.688925, 2.530019",01/02/2003
2,9120001,Musée du Château,2019,ILE-DE-France,DOURDAN,01/02/2003,4476,6384,10860,NaN,...,Place du général de Gaulle,Château de Dourdan,91410,01 64 59 66 83,www.dourdan.fr/a-voir-a-vivre/lechateauetsonmu...,48.529685,2.011803,9120001.0,"48.529685, 2.011803",01/02/2003
3,9120001,Musée du Château,2020,ILE-DE-France,DOURDAN,01/02/2003,9244,0,9244,NaN,...,Place du général de Gaulle,Château de Dourdan,91410,01 64 59 66 83,www.dourdan.fr/a-voir-a-vivre/lechateauetsonmu...,48.529685,2.011803,9120001.0,"48.529685, 2.011803",01/02/2003
4,9201202,Musée-Jardin Paul Landowski,2019,ILE-DE-France,BOULOGNE-BILLANCOURT,01/02/2003,0,0,0,NaN,...,"14, rue Max-Blondat",NaN,92100,01 55 1 8 46 42,www.boulognebillancourt.com/loisirs/culture/le...,48.844261,2.246847,9201202.0,"48.844261, 2.246847",01/02/2003


In [36]:
len(df)

261

In [37]:
df['REGION'].unique()

array(['ILE-DE-France', 'BRETAGNE', 'NOUVELLE-AQUITAINE',
       "PROVENCE-ALPES-CÔTE D'AZUR"], dtype=object)

In [38]:
Paris_museums = df.loc[df['VILLE'] == 'PARIS']

In [39]:
len(Paris_museums)

99

In [40]:
Paris_museums.isna().sum()

REF DU MUSEE                            0
NOM DU MUSEE                            0
Année                                   0
REGION                                  0
VILLE                                   0
DATE APPELLATION                        2
PAYANT                                  0
GRATUIT                                 0
TOTAL                                   0
Note                                   94
ID MUSEOFILE                            0
NOMDEP                                  0
Observations                           84
Région administrative                   0
Département                             0
Identifiant Muséofile                   0
Commune                                 0
Nom officiel du musée                   0
Adresse                                 8
Lieu                                   44
Code Postal                             0
Téléphone                               6
URL                                     2
Latitude                          

In [41]:
#Too many missing values so we drop those columns

Paris_museums.drop(columns=['Note', 'Observations'], inplace=True)

/var/folders/xp/jp6pr1sn3p50vy46h4q0ntqr0000gn/T/ipykernel_60167/2214035505.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Paris_museums.drop(columns=['Note', 'Observations'], inplace=True)


In [42]:
Paris_museums.isna().sum()

REF DU MUSEE                            0
NOM DU MUSEE                            0
Année                                   0
REGION                                  0
VILLE                                   0
DATE APPELLATION                        2
PAYANT                                  0
GRATUIT                                 0
TOTAL                                   0
ID MUSEOFILE                            0
NOMDEP                                  0
Région administrative                   0
Département                             0
Identifiant Muséofile                   0
Commune                                 0
Nom officiel du musée                   0
Adresse                                 8
Lieu                                   44
Code Postal                             0
Téléphone                               6
URL                                     2
Latitude                                0
Longitude                               0
REF_Deps                          

In [43]:
#Paris_museums.loc[Paris_museums['Lieu'].isna()]

In [44]:
#We could drop the 'Lieu' column since we already have the address, and fill in the 8 addresses missing.
#We could fill in the 6 phone numbers missing (if we use this column), as well as the 2 URL missing.
#We could also drop the 'DATE APPELLATION' since we already have the 'Date arrêté attribution appellation' (that could be removed as well since we might not use it).
#We could also remove the 'Commune' since we already have the 'Code Postal'.
#We could also remove either the Latitude and Longitude columns or the geolocalisation one since those two both give the coordinates of the museums.
#We should also remove duplicates columns (e.g. nom du musée or Nom officiel du musée, REGION or Région administrative, NOMDEP or Département, etc.)

In [45]:
Paris_museums.drop(columns=['DATE APPELLATION', 'Région administrative', 'Département', 'Nom officiel du musée', 'Lieu', 'Commune', 'REF_Deps' ], inplace=True)

/var/folders/xp/jp6pr1sn3p50vy46h4q0ntqr0000gn/T/ipykernel_60167/817404108.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Paris_museums.drop(columns=['DATE APPELLATION', 'Région administrative', 'Département', 'Nom officiel du musée', 'Lieu', 'Commune', 'REF_Deps' ], inplace=True)


In [46]:
Paris_museums.loc[Paris_museums['Adresse'].isna()]

,REF DU MUSEE,NOM DU MUSEE,Année,REGION,VILLE,PAYANT,GRATUIT,TOTAL,ID MUSEOFILE,NOMDEP,Identifiant Muséofile,Adresse,Code Postal,Téléphone,URL,Latitude,Longitude,geolocalisation,Date arrêté attribution appellation
24,7510519,Institut du Monde Arabe,2019,ILE-DE-France,PARIS,238591,1191,239782,M0365,PARIS,M0365,NaN,75005,NaN,www.imarabe.org/fr/musee/presentation-du-musee,48.843955,2.352176,"48.843955, 2.352176",21/03/2011
25,7510519,Institut du Monde Arabe,2020,ILE-DE-France,PARIS,78858,0,78858,M0365,PARIS,M0365,NaN,75005,NaN,www.imarabe.org/fr/musee/presentation-du-musee,48.843955,2.352176,"48.843955, 2.352176",21/03/2011
140,7510103,Établissement public du musée d'Orsay et du mu...,2020,ILE-DE-France,PARIS,119888,111268,231156,M5030,PARIS,M5030,NaN,75001,01 40 20 67 71,www.musee-orangerie.fr/,48.868053,2.326622,"48.868053, 2.326622",06/01/2002
141,7510103,Etablissement public du musée d'Orsay et de l'...,2019,ILE-DE-France,PARIS,654134,375791,1029925,M5030,PARIS,M5030,NaN,75001,01 40 20 67 71,www.musee-orangerie.fr/,48.868053,2.326622,"48.868053, 2.326622",06/01/2002
168,7511202,Établissement Public de la Porte Dorée - (MNHI...,2019,ILE-DE-France,PARIS,224318,301276,525594,M5084,PARIS,M5084,NaN,75012,01 53 59 58 60,www.histoire-immigration.fr/,48.836753,2.390394,"48.836753, 2.390394",17/11/2006
169,7511202,Établissement Public de la Porte Dorée - (MNHI...,2020,ILE-DE-France,PARIS,108642,111070,219712,M5084,PARIS,M5084,NaN,75012,01 53 59 58 60,www.histoire-immigration.fr/,48.836753,2.390394,"48.836753, 2.390394",17/11/2006
208,7510610,Musée de l'Ecole Nationale Supérieure des Beau...,2020,ILE-DE-France,PARIS,4996,3664,8660,M5051,PARIS,M5051,NaN,75006,NaN,beauxartsparis.fr/fr/collection/presentation,48.845748,2.331670,"48.845748, 2.33167",10/07/2017
209,7510610,Musée de l'Ecole Nationale Supérieure des Beau...,2019,ILE-DE-France,PARIS,0,59113,59113,M5051,PARIS,M5051,NaN,75006,NaN,beauxartsparis.fr/fr/collection/presentation,48.845748,2.331670,"48.845748, 2.33167",10/07/2017


In [47]:
Paris_museums.reset_index()

,index,REF DU MUSEE,NOM DU MUSEE,Année,REGION,VILLE,PAYANT,GRATUIT,TOTAL,ID MUSEOFILE,NOMDEP,Identifiant Muséofile,Adresse,Code Postal,Téléphone,URL,Latitude,Longitude,geolocalisation,Date arrêté attribution appellation
0,8,7511503,Musée de la Libération de Paris - Musée du Gén...,2019,ILE-DE-France,PARIS,0,49570,49570,M1109,PARIS,M1109,23 allée de la 2ème Division Blindée,75015,01 40 64 39 44,www.museeliberation-leclerc-moulin.paris.fr/,48.840614,2.319773,"48.840614, 2.319773",08/07/2004
1,9,7511503,Musée de la Libération de Paris - Musée du Gén...,2020,ILE-DE-France,PARIS,10232,31075,41307,M1109,PARIS,M1109,23 allée de la 2ème Division Blindée,75015,01 40 64 39 44,www.museeliberation-leclerc-moulin.paris.fr/,48.840614,2.319773,"48.840614, 2.319773",08/07/2004
2,10,7510705,Etablissement public du musée d'Orsay et de l'...,2019,ILE-DE-France,PARIS,2010889,1640727,3651616,M5060,PARIS,M5060,"1, rue de Bellechasse",75007,01 40 49 48 14,www.musee-orsay.fr/,48.859157,2.323672,"48.859157, 2.323672",06/01/2002
3,11,7510705,Établissement public du musée d'Orsay et du mu...,2020,ILE-DE-France,PARIS,424192,443082,867274,M5060,PARIS,M5060,"1, rue de Bellechasse",75007,01 40 49 48 14,www.musee-orsay.fr/,48.859157,2.323672,"48.859157, 2.323672",06/01/2002
4,12,7510516,Musée du Service de Santé des Armées du Val-de...,2019,ILE-DE-France,PARIS,9508,4475,13983,M9024,PARIS,M9024,"1, place Alphonse Laveran",75005,01 40 51 51 92,www.ecole-valdegrace.sante.defense.gouv.fr/bib...,48.840814,2.341091,"48.840814, 2.341091",13/12/2006
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
94,247,7511603,Etablissement Public du Musée des Arts Asiatiq...,2020,ILE-DE-France,PARIS,45578,50758,96336,M5005,PARIS,M5005,"6, place d'Iéna",75116,01 56 52 53 00,www.museeguimet.fr,48.864250,2.293754,"48.86425, 2.293754",06/01/2002
95,248,7510807,"Musée Cernuschi, Musée des Arts de l'Asie de l...",2019,ILE-DE-France,PARIS,16917,29111,46028,M1106,PARIS,M1106,7 avenue Vélasquez,75008,01 53 96 21 50,www.cernuschi.paris.fr/,48.879693,2.312194,"48.879693, 2.312194",08/07/2004
96,249,7510807,"Musée Cernuschi, Musée des Arts de l'Asie de l...",2020,ILE-DE-France,PARIS,2859,23251,26110,M1106,PARIS,M1106,7 avenue Vélasquez,75008,01 53 96 21 50,www.cernuschi.paris.fr/,48.879693,2.312194,"48.879693, 2.312194",08/07/2004
97,250,7510609,Musée du 11 Conti - Monnaie de Paris,2019,ILE-DE-France,PARIS,96200,33800,130000,M5066,PARIS,M5066,"11, quai de Conti",75006,01 40 46 57 57,www.monnaiedeparis.fr/,48.857003,2.338699,"48.857003, 2.338699",06/01/2002


Idée : afficher les 99 parisiens et leur fréquentation sous forme de mapchart ou stackedbarchart par exemple

## Merging exhibitions & parisian museums -> 59 rows, see if it can be relevant for one analysis or not

In [48]:
Paris_museums.head()

,REF DU MUSEE,NOM DU MUSEE,Année,REGION,VILLE,PAYANT,GRATUIT,TOTAL,ID MUSEOFILE,NOMDEP,Identifiant Muséofile,Adresse,Code Postal,Téléphone,URL,Latitude,Longitude,geolocalisation,Date arrêté attribution appellation
8,7511503,Musée de la Libération de Paris - Musée du Gén...,2019,ILE-DE-France,PARIS,0,49570,49570,M1109,PARIS,M1109,23 allée de la 2ème Division Blindée,75015,01 40 64 39 44,www.museeliberation-leclerc-moulin.paris.fr/,48.840614,2.319773,"48.840614, 2.319773",08/07/2004
9,7511503,Musée de la Libération de Paris - Musée du Gén...,2020,ILE-DE-France,PARIS,10232,31075,41307,M1109,PARIS,M1109,23 allée de la 2ème Division Blindée,75015,01 40 64 39 44,www.museeliberation-leclerc-moulin.paris.fr/,48.840614,2.319773,"48.840614, 2.319773",08/07/2004
10,7510705,Etablissement public du musée d'Orsay et de l'...,2019,ILE-DE-France,PARIS,2010889,1640727,3651616,M5060,PARIS,M5060,"1, rue de Bellechasse",75007,01 40 49 48 14,www.musee-orsay.fr/,48.859157,2.323672,"48.859157, 2.323672",06/01/2002
11,7510705,Établissement public du musée d'Orsay et du mu...,2020,ILE-DE-France,PARIS,424192,443082,867274,M5060,PARIS,M5060,"1, rue de Bellechasse",75007,01 40 49 48 14,www.musee-orsay.fr/,48.859157,2.323672,"48.859157, 2.323672",06/01/2002
12,7510516,Musée du Service de Santé des Armées du Val-de...,2019,ILE-DE-France,PARIS,9508,4475,13983,M9024,PARIS,M9024,"1, place Alphonse Laveran",75005,01 40 51 51 92,www.ecole-valdegrace.sante.defense.gouv.fr/bib...,48.840814,2.341091,"48.840814, 2.341091",13/12/2006


In [49]:
exhibitions.head()

,ID,URL,Titre,Chapeau,Description,Date de début,Date de fin,Occurrences,Description de la date,URL de l'image,...,Date de mise à jour,Image de couverture,Programmes,En ligne - address_url,En ligne - address_url_text,En ligne - address_text,title_event,audience,childrens,contributor_group
0,34155,https://www.paris.fr/evenements/baila-babibou-...,"Baila Babibou : spectacle danse & musique, dès...",Baila Babibou propose un dialogue entre une mé...,<p>La danseuse démarre le spectacle en chrysal...,2023-06-21T13:30:00+02:00,2023-06-21T14:00:00+02:00,2023-06-21T11:30:00+02:00_2023-06-21T12:00:00+...,Le mercredi 21 juin 2023<br />de 10h30 à 11h00...,https://cdn.paris.fr/qfapv4/2023/03/17/huge-06...,...,2023-03-17T17:30:43+01:00,https://opendata.paris.fr/api/explore/v2.1/cat...,NaN,NaN,NaN,NaN,NaN,Public enfants. A partir de 1 ans. Jusqu'à 7 ans.,NaN,Bibliothèques
1,34268,https://www.paris.fr/evenements/le-fil-des-noe...,Le fil des nœuds,Exposition collective de La.ima du 10 juin au ...,<p>Qu'y a-t-il au-delà de la peau qui cache la...,2023-06-10T02:00:00+02:00,2023-07-09T01:59:59+02:00,NaN,Du samedi 10 juin 2023 au samedi 08 juillet 20...,https://cdn.paris.fr/qfapv4/2023/03/19/huge-04...,...,2023-03-19T15:06:21+01:00,https://opendata.paris.fr/api/explore/v2.1/cat...,NaN,NaN,NaN,NaN,David Tanné,Public adolescents adultes.,NaN,Bibliothèques
2,34245,https://www.paris.fr/evenements/parcours-d-int...,Parcours d’intellectuels en exil : un humanism...,Un cycle de 4 conférences dédié aux trajectoir...,<p>L’exil n’a jamais épargné les intellectuels...,2023-03-28T02:00:00+02:00,2023-06-14T01:59:59+02:00,NaN,Du mardi 28 mars 2023 au mardi 13 juin 2023 : ...,https://cdn.paris.fr/qfapv4/2023/03/18/huge-e3...,...,2023-03-20T09:56:18+01:00,https://opendata.paris.fr/api/explore/v2.1/cat...,NaN,NaN,NaN,NaN,NaN,Public adolescents adultes.,Conférence : Celso Furtado : un exil académiqu...,Internautes
3,33666,https://www.paris.fr/evenements/concert-java-m...,"Concert : Java, musique du pays Sunda",Venez découvrir la musique du pays Sunda de l'...,"<p>Basé à Paris, <strong>Gamelan Degung Sora G...",2023-05-26T22:00:00+02:00,2023-05-27T00:00:00+02:00,2023-05-26T20:00:00+02:00_2023-05-26T22:00:00+...,Le vendredi 26 mai 2023<br />de 19h00 à 21h00<...,https://cdn.paris.fr/qfapv4/2023/03/09/huge-7f...,...,2023-03-22T11:02:35+01:00,https://opendata.paris.fr/api/explore/v2.1/cat...,NaN,NaN,NaN,NaN,NaN,Tout public.,NaN,Bibliothèques
4,34251,https://www.paris.fr/evenements/p-tit-dej-litt...,P'tit-Déj' Littéraire,Les bibliothécaires vous lisent leurs derniers...,"<p>Depuis quelques éditions, cette animation e...",2023-06-10T13:30:00+02:00,2023-06-10T15:00:00+02:00,2023-06-10T11:30:00+02:00_2023-06-10T13:00:00+...,Le samedi 10 juin 2023<br />de 10h30 à 12h00<b...,https://cdn.paris.fr/qfapv4/2023/03/04/huge-a3...,...,2023-03-22T13:45:16+01:00,https://opendata.paris.fr/api/explore/v2.1/cat...,NaN,NaN,NaN,NaN,NaN,Public adolescents adultes.,NaN,Bibliothèques


In [50]:
Paris_exhibitions = exhibitions.merge(Paris_museums, left_on="Nom du lieu", right_on="NOM DU MUSEE", how='left')
Paris_exhibitions.head()

,ID,URL_x,Titre,Chapeau,Description,Date de début,Date de fin,Occurrences,Description de la date,URL de l'image,...,NOMDEP,Identifiant Muséofile,Adresse,Code Postal,Téléphone,URL_y,Latitude,Longitude,geolocalisation,Date arrêté attribution appellation
0,34155,https://www.paris.fr/evenements/baila-babibou-...,"Baila Babibou : spectacle danse & musique, dès...",Baila Babibou propose un dialogue entre une mé...,<p>La danseuse démarre le spectacle en chrysal...,2023-06-21T13:30:00+02:00,2023-06-21T14:00:00+02:00,2023-06-21T11:30:00+02:00_2023-06-21T12:00:00+...,Le mercredi 21 juin 2023<br />de 10h30 à 11h00...,https://cdn.paris.fr/qfapv4/2023/03/17/huge-06...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,34268,https://www.paris.fr/evenements/le-fil-des-noe...,Le fil des nœuds,Exposition collective de La.ima du 10 juin au ...,<p>Qu'y a-t-il au-delà de la peau qui cache la...,2023-06-10T02:00:00+02:00,2023-07-09T01:59:59+02:00,NaN,Du samedi 10 juin 2023 au samedi 08 juillet 20...,https://cdn.paris.fr/qfapv4/2023/03/19/huge-04...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,34245,https://www.paris.fr/evenements/parcours-d-int...,Parcours d’intellectuels en exil : un humanism...,Un cycle de 4 conférences dédié aux trajectoir...,<p>L’exil n’a jamais épargné les intellectuels...,2023-03-28T02:00:00+02:00,2023-06-14T01:59:59+02:00,NaN,Du mardi 28 mars 2023 au mardi 13 juin 2023 : ...,https://cdn.paris.fr/qfapv4/2023/03/18/huge-e3...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,33666,https://www.paris.fr/evenements/concert-java-m...,"Concert : Java, musique du pays Sunda",Venez découvrir la musique du pays Sunda de l'...,"<p>Basé à Paris, <strong>Gamelan Degung Sora G...",2023-05-26T22:00:00+02:00,2023-05-27T00:00:00+02:00,2023-05-26T20:00:00+02:00_2023-05-26T22:00:00+...,Le vendredi 26 mai 2023<br />de 19h00 à 21h00<...,https://cdn.paris.fr/qfapv4/2023/03/09/huge-7f...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,34251,https://www.paris.fr/evenements/p-tit-dej-litt...,P'tit-Déj' Littéraire,Les bibliothécaires vous lisent leurs derniers...,"<p>Depuis quelques éditions, cette animation e...",2023-06-10T13:30:00+02:00,2023-06-10T15:00:00+02:00,2023-06-10T11:30:00+02:00_2023-06-10T13:00:00+...,Le samedi 10 juin 2023<br />de 10h30 à 12h00<b...,https://cdn.paris.fr/qfapv4/2023/03/04/huge-a3...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [51]:
#Paris_exhibitions.loc[Paris_exhibitions["NOM DU MUSEE"].notna()].count() 

#See if it can be useful, 59 exhibitions with all the infos (museums, visitors, exhibitions)

In [52]:
Paris_exhibitions.isna().sum()

ID                                        0
URL_x                                     0
Titre                                     0
Chapeau                                   5
Description                              19
                                       ... 
URL_y                                  2045
Latitude                               2045
Longitude                              2045
geolocalisation                        2045
Date arrêté attribution appellation    2045
Length: 61, dtype: int64

In [53]:
Paris_exhibitions.loc[Paris_exhibitions["NOM DU MUSEE"].notna()]

,ID,URL_x,Titre,Chapeau,Description,Date de début,Date de fin,Occurrences,Description de la date,URL de l'image,...,NOMDEP,Identifiant Muséofile,Adresse,Code Postal,Téléphone,URL_y,Latitude,Longitude,geolocalisation,Date arrêté attribution appellation
26,35898,https://www.paris.fr/evenements/nouveaute-baby...,Nouveauté ! Baby-visite au musée Bourdelle,Profitez d'une visite sensorielle et d'un temp...,<p>Une \nvisite sensorielle et participative a...,2023-05-10T13:30:00+02:00,2023-06-14T14:30:00+02:00,2023-06-14T11:30:00+02:00_2023-06-14T12:30:00+...,Le mercredi 14 juin 2023<br />de 10h30 à 11h30...,https://cdn.paris.fr/qfapv4/2023/04/26/huge-00...,...,PARIS,M1103,16 rue Antoine Bourdelle,75015.0,01 49 54 73 73,www.bourdelle.paris.fr/,48.843017,2.318830,"48.843017, 2.31883",08/07/2004
27,35898,https://www.paris.fr/evenements/nouveaute-baby...,Nouveauté ! Baby-visite au musée Bourdelle,Profitez d'une visite sensorielle et d'un temp...,<p>Une \nvisite sensorielle et participative a...,2023-05-10T13:30:00+02:00,2023-06-14T14:30:00+02:00,2023-06-14T11:30:00+02:00_2023-06-14T12:30:00+...,Le mercredi 14 juin 2023<br />de 10h30 à 11h30...,https://cdn.paris.fr/qfapv4/2023/04/26/huge-00...,...,PARIS,M1103,16 rue Antoine Bourdelle,75015.0,01 49 54 73 73,www.bourdelle.paris.fr/,48.843017,2.318830,"48.843017, 2.31883",08/07/2004
34,36253,https://www.paris.fr/evenements/mini-sculpteur...,Mini-sculpteur 1-3 ans en famille,Les tout-petits et leurs parents sont invités ...,<p>Après\n une découverte ludique du musée Bou...,2023-07-08T18:30:00+02:00,2023-09-03T14:00:00+02:00,2023-07-08T16:30:00+02:00_2023-07-08T17:30:00+...,Le dimanche 03 septembre 2023<br />de 10h00 à ...,https://cdn.paris.fr/qfapv4/2023/05/04/huge-b3...,...,PARIS,M1103,16 rue Antoine Bourdelle,75015.0,01 49 54 73 73,www.bourdelle.paris.fr/,48.843017,2.318830,"48.843017, 2.31883",08/07/2004
35,36253,https://www.paris.fr/evenements/mini-sculpteur...,Mini-sculpteur 1-3 ans en famille,Les tout-petits et leurs parents sont invités ...,<p>Après\n une découverte ludique du musée Bou...,2023-07-08T18:30:00+02:00,2023-09-03T14:00:00+02:00,2023-07-08T16:30:00+02:00_2023-07-08T17:30:00+...,Le dimanche 03 septembre 2023<br />de 10h00 à ...,https://cdn.paris.fr/qfapv4/2023/05/04/huge-b3...,...,PARIS,M1103,16 rue Antoine Bourdelle,75015.0,01 49 54 73 73,www.bourdelle.paris.fr/,48.843017,2.318830,"48.843017, 2.31883",08/07/2004
71,36561,https://www.paris.fr/evenements/atelier-de-mod...,Atelier de modelage en famille - Les étranges ...,Après une découverte joyeuse des jardins du mu...,<p>Après une découverte joyeuse des jardins du...,2023-07-26T19:00:00+02:00,2023-08-10T20:30:00+02:00,2023-07-26T17:00:00+02:00_2023-07-26T18:30:00+...,Le jeudi 10 août 2023<br />de 16h00 à 17h30<br...,https://cdn.paris.fr/qfapv4/2023/05/10/huge-d2...,...,PARIS,M1103,16 rue Antoine Bourdelle,75015.0,01 49 54 73 73,www.bourdelle.paris.fr/,48.843017,2.318830,"48.843017, 2.31883",08/07/2004
72,36561,https://www.paris.fr/evenements/atelier-de-mod...,Atelier de modelage en famille - Les étranges ...,Après une découverte joyeuse des jardins du mu...,<p>Après une découverte joyeuse des jardins du...,2023-07-26T19:00:00+02:00,2023-08-10T20:30:00+02:00,2023-07-26T17:00:00+02:00_2023-07-26T18:30:00+...,Le jeudi 10 août 2023<br />de 16h00 à 17h30<br...,https://cdn.paris.fr/qfapv4/2023/05/10/huge-d2...,...,PARIS,M1103,16 rue Antoine Bourdelle,75015.0,01 49 54 73 73,www.bourdelle.paris.fr/,48.843017,2.318830,"48.843017, 2.31883",08/07/2004
73,36559,https://www.paris.fr/evenements/atelier-de-mod...,Atelier de modelage pour enfants - Mon héros o...,"Modelez votre héros Olympique, dans le sillage...",<p>Après une observation des déclinaisons d'<e...,2023-07-20T13:00:00+02:00,2023-07-27T15:00:00+02:00,2023-07-20T11:00:00+02:00_2023-07-20T13:00:00+...,Le jeudi 27 juillet 2023<br />de 10h00 à 12h00...,https://cdn.paris.fr/qfapv4/2023/05/10/huge-e1...,...,PARIS,M1103,16 rue Antoine Bourdelle,75015.0,01 49 54 73 73,www.bourdelle.par

In [54]:
len(Paris_exhibitions)

2104

In [55]:
Paris_exhibitions.drop(columns=['ID', "Texte alternatif de l'image", "Crédit de l'image", 'Accès PMR', 'Accès mal voyant', 'Accès mal entendant', 'Transport', 'Url de contact', 'Téléphone de contact', 'Email de contact', 'En ligne - address_url', 'En ligne - address_url_text', 'En ligne - address_text', 'title_event', 'childrens', 'URL Facebook associée', 'URL Twitter associée', 'URL de réservation', 'URL de réservation - Texte', 'Programmes', 'Détail du prix', 'Occurrences', 'Description de la date', "URL de l'image", "Type d'accès", "Date de mise à jour", "Image de couverture", "contributor_group", "REF DU MUSEE", "NOM DU MUSEE", "VILLE", "ID MUSEOFILE", "Identifiant Muséofile", "Adresse", "Code Postal", "Téléphone", "URL_y", "Latitude", "Longitude", "geolocalisation", "Date arrêté attribution appellation"], inplace=True)

In [56]:
Paris_exhibitions.head()

,URL_x,Titre,Chapeau,Description,Date de début,Date de fin,Mots clés,Nom du lieu,Adresse du lieu,Code postal,Ville,Coordonnées géographiques,Type de prix,audience,Année,REGION,PAYANT,GRATUIT,TOTAL,NOMDEP
0,https://www.paris.fr/evenements/baila-babibou-...,"Baila Babibou : spectacle danse & musique, dès...",Baila Babibou propose un dialogue entre une mé...,<p>La danseuse démarre le spectacle en chrysal...,2023-06-21T13:30:00+02:00,2023-06-21T14:00:00+02:00,"Danse,Enfants,Spectacle musical",Bibliothèque Jacqueline de Romilly,16 avenue de la Porte Montmartre,75018,Paris,"48.8994220173449, 2.33641997088512",gratuit,Public enfants. A partir de 1 ans. Jusqu'à 7 ans.,NaN,NaN,NaN,NaN,NaN,NaN
1,https://www.paris.fr/evenements/le-fil-des-noe...,Le fil des nœuds,Exposition collective de La.ima du 10 juin au ...,<p>Qu'y a-t-il au-delà de la peau qui cache la...,2023-06-10T02:00:00+02:00,2023-07-09T01:59:59+02:00,"Art contemporain,Expo,Photo",Médiathèque Edmond Rostand,11 rue Nicolas Chuquet,75017,Paris,"48.8883939892465, 2.30309000346023",gratuit,Public adolescents adultes.,NaN,NaN,NaN,NaN,NaN,NaN
2,https://www.paris.fr/evenements/parcours-d-int...,Parcours d’intellectuels en exil : un humanism...,Un cycle de 4 conférences dédié aux trajectoir...,<p>L’exil n’a jamais épargné les intellectuels...,2023-03-28T02:00:00+02:00,2023-06-14T01:59:59+02:00,"Conférence,Littérature,Sciences",Fondation Maison des sciences de l'homme,54 boulevard Raspail,75006,Paris,"48.8502890192206, 2.32696501646515",gratuit,Public adolescents adultes.,NaN,NaN,NaN,NaN,NaN,NaN
3,https://www.paris.fr/evenements/concert-java-m...,"Concert : Java, musique du pays Sunda",Venez découvrir la musique du pays Sunda de l'...,"<p>Basé à Paris, <strong>Gamelan Degung Sora G...",2023-05-26T22:00:00+02:00,2023-05-27T00:00:00+02:00,"Concert,Musique",Médiathèque Marguerite Yourcenar,"41, rue d'Alleray",75015,Paris,"48.83680697942, 2.30348101924335",gratuit,Tout public.,NaN,NaN,NaN,NaN,NaN,NaN
4,https://www.paris.fr/evenements/p-tit-dej-litt...,P'tit-Déj' Littéraire,Les bibliothécaires vous lisent leurs derniers...,"<p>Depuis quelques éditions, cette animation e...",2023-06-10T13:30:00+02:00,2023-06-10T15:00:00+02:00,Littérature,Médiathèque Marguerite Yourcenar,"41, rue d'Alleray",75015,Paris,"48.83680697942, 2.30348101924335",gratuit,Public adolescents adultes.,NaN,NaN,NaN,NaN,NaN,NaN


## Exhibitions keywords analysis & cleaning

In [57]:
Paris_exhibitions['Mots clés'].unique()

array(['Danse,Enfants,Spectacle musical', 'Art contemporain,Expo,Photo',
       'Conférence,Littérature,Sciences', 'Concert,Musique',
       'Littérature', 'Expo,Street-art', 'Art contemporain,Enfants,Expo',
       'Expo', 'Cirque,Conférence', 'Enfants,Musique,Spectacle musical',
       'Atelier', 'Atelier,Peinture', 'Atelier,Cinéma,Photo',
       'Conférence,Histoire,Sport', 'Art contemporain,Expo,Peinture',
       'Concert,Musique,Spectacle musical', 'Atelier,Enfants,Nature',
       'Atelier,Danse,Solidarité', 'Jeux 2024,Loisirs', 'Jeux 2024,Sport',
       'Atelier,Enfants,Loisirs', 'Spectacle musical,Street-art,Théâtre',
       'Expo,Histoire,Photo', 'Atelier,Enfants', 'Cinéma', 'Nature',
       'Atelier,Enfants,Innovation', 'Atelier,Loisirs,Nature',
       'Conférence', 'Balade,Histoire', 'Art contemporain,installation',
       'musique', 'installation,musique', 'Enfants,Théâtre',
       'Art contemporain,Expo', 'Art contemporain,Expo,Nature',
       'Art contemporain,exposition',


In [58]:
len(Paris_exhibitions.loc[Paris_exhibitions['Mots clés'].isna()])

#since there are 52 rows (out of 2104) that don't have any key words, we're gonna drop them

52

In [59]:
Paris_exhibitions.dropna(subset=['Mots clés'], inplace=True)

In [60]:
#since there are dates instead of real key words for describing exhibitions, we're gonna have a look at these records

Paris_exhibitions.loc[Paris_exhibitions['Mots clés'].str.startswith('2')]

,URL_x,Titre,Chapeau,Description,Date de début,Date de fin,Mots clés,Nom du lieu,Adresse du lieu,Code postal,Ville,Coordonnées géographiques,Type de prix,audience,Année,REGION,PAYANT,GRATUIT,TOTAL,NOMDEP
51,2023-11-30T13:30:47+01:00_2023-11-30T18:00:47+...,2023-12-01T13:30:47+01:00_2023-12-01T18:00:47+...,2023-12-02T13:30:47+01:00_2023-12-02T18:00:47+...,2023-12-03T13:30:47+01:00_2023-12-03T18:00:47+...,2023-12-06T13:30:47+01:00_2023-12-06T18:00:47+...,2023-12-07T13:30:47+01:00_2023-12-07T18:00:47+...,2023-12-15T13:30:47+01:00_2023-12-15T18:00:47+...,2023-12-16T13:30:47+01:00_2023-12-16T18:00:47+...,2023-12-17T13:30:47+01:00_2023-12-17T18:00:47+...,2023-12-20T13:30:47+01:00_2023-12-20T18:00:47+...,2023-12-21T13:30:47+01:00_2023-12-21T18:00:47+...,2023-12-22T13:30:47+01:00_2023-12-22T18:00:47+...,Voute sculptée de l'escalier princier de la to...,http://www.tourjeansanspeur.com,NaN,NaN,NaN,NaN,NaN,NaN
680,2023-11-17T10:00:03+01:00_2023-11-17T17:00:03+...,2023-11-18T10:00:03+01:00_2023-11-18T17:00:03+...,2023-11-18T10:00:03+01:00_2023-11-18T17:00:03+...,2023-11-19T10:00:03+01:00_2023-11-19T17:00:03+...,2023-11-19T10:00:03+01:00_2023-11-19T17:00:03+...,2023-11-20T10:00:03+01:00_2023-11-20T17:00:03+...,2023-11-23T10:00:03+01:00_2023-11-23T17:00:03+...,2023-11-23T10:00:03+01:00_2023-11-23T17:00:03+...,2023-11-24T10:00:03+01:00_2023-11-24T17:00:03+...,2023-11-24T10:00:03+01:00_2023-11-24T17:00:03+...,2023-11-25T10:00:03+01:00_2023-11-25T17:00:03+...,2023-11-25T10:00:03+01:00_2023-11-25T17:00:03+...,2023-11-30T10:00:03+01:00_2023-11-30T17:00:03+...,2023-12-06T10:00:03+01:00_2023-12-06T17:00:03+...,NaN,NaN,NaN,NaN,NaN,NaN
1211,2023-06-08T11:00:54+02:00_2023-06-08T14:00:54+...,2023-06-09T11:00:54+02:00_2023-06-09T14:00:54+...,2023-06-10T10:30:54+02:00_2023-06-10T19:00:54+...,2023-06-11T10:30:54+02:00_2023-06-11T19:00:54+...,2023-06-13T11:00:54+02:00_2023-06-13T14:00:54+...,2023-06-15T11:00:54+02:00_2023-06-15T14:00:54+...,2023-06-23T11:00:54+02:00_2023-06-23T14:00:54+...,2023-06-24T10:30:54+02:00_2023-06-24T19:00:54+...,2023-06-25T10:30:54+02:00_2023-06-25T19:00:54+...,2023-06-27T11:00:54+02:00_2023-06-27T14:00:54+...,2023-06-29T11:00:54+02:00_2023-06-29T14:00:54+...,2023-06-30T11:00:54+02:00_2023-06-30T14:00:54+...,2023-07-14T11:00:54+02:00_2023-07-14T14:00:54+...,2023-07-30T10:30:54+02:00_2023-07-30T19:00:54+...,NaN,NaN,NaN,NaN,NaN,NaN
1574,2023-07-21T12:00:00+02:00_2023-07-21T21:00:00+...,2023-07-22T12:00:00+02:00_2023-07-22T21:00:00+...,2023-07-23T12:00:00+02:00_2023-07-23T21:00:00+...,2023-07-24T12:00:00+02:00_2023-07-24T21:00:00+...,2023-07-25T12:00:00+02:00_2023-07-25T21:00:00+...,2023-07-26T12:00:00+02:00_2023-07-26T21:00:00+...,2023-08-01T12:00:00+02:00_2023-08-01T21:00:00+...,2023-08-02T12:00:00+02:00_2023-08-02T21:00:00+...,2023-08-03T12:00:00+02:00_2023-08-03T21:00:00+...,2023-08-04T12:00:00+02:00_2023-08-04T21:00:00+...,2023-08-05T12:00:00+02:00_2023-08-05T21:00:00+...,2023-08-06T12:00:00+02:00_2023-08-06T21:00:00+...,2023-08-16T12:00:00+02:00_2023-08-16T21:00:00+...,2023-08-28T12:00:00+02:00_2023-08-28T21:00:00+...,NaN,NaN,NaN,NaN,NaN,NaN


In [61]:
#these records don't add any values so we're gonna remove them

Paris_exhibitions.drop(Paris_exhibitions.loc[Paris_exhibitions['Mots clés'].str.startswith('2')].index, inplace=True)

In [62]:
Paris_exhibitions['Mots clés'].unique()

array(['Danse,Enfants,Spectacle musical', 'Art contemporain,Expo,Photo',
       'Conférence,Littérature,Sciences', 'Concert,Musique',
       'Littérature', 'Expo,Street-art', 'Art contemporain,Enfants,Expo',
       'Expo', 'Cirque,Conférence', 'Enfants,Musique,Spectacle musical',
       'Atelier', 'Atelier,Peinture', 'Atelier,Cinéma,Photo',
       'Conférence,Histoire,Sport', 'Art contemporain,Expo,Peinture',
       'Concert,Musique,Spectacle musical', 'Atelier,Enfants,Nature',
       'Atelier,Danse,Solidarité', 'Jeux 2024,Loisirs', 'Jeux 2024,Sport',
       'Atelier,Enfants,Loisirs', 'Spectacle musical,Street-art,Théâtre',
       'Expo,Histoire,Photo', 'Atelier,Enfants', 'Cinéma', 'Nature',
       'Atelier,Enfants,Innovation', 'Atelier,Loisirs,Nature',
       'Conférence', 'Balade,Histoire', 'Art contemporain,installation',
       'musique', 'installation,musique', 'Enfants,Théâtre',
       'Art contemporain,Expo', 'Art contemporain,Expo,Nature',
       'Art contemporain,exposition',


In [63]:
#looking at the unique keywords we have, we still have one weird value that we're gonna remove

Paris_exhibitions.loc[Paris_exhibitions['Mots clés'] == "A l'Aquarium tropical de Paris, le Spot est un mur numérique qui propose des rencontres inédites avec les baleines."]

,URL_x,Titre,Chapeau,Description,Date de début,Date de fin,Mots clés,Nom du lieu,Adresse du lieu,Code postal,Ville,Coordonnées géographiques,Type de prix,audience,Année,REGION,PAYANT,GRATUIT,TOTAL,NOMDEP
422,2025-01-17T10:00:00+01:00_2025-01-17T17:30:00+...,2025-01-18T10:00:00+01:00_2025-01-18T19:00:00+...,2025-01-19T10:00:00+01:00_2025-01-19T19:00:00+...,2025-01-21T10:00:00+01:00_2025-01-21T17:30:00+...,2025-01-22T10:00:00+01:00_2025-01-22T17:30:00+...,2025-01-23T10:00:00+01:00_2025-01-23T17:30:00+...,"A l'Aquarium tropical de Paris, le Spot est un...",Aquarium tropical de Paris,"Enfants,Expo",L'Aquarium tropical de la porte Dorée,293 avenue Daumesnil,75 012,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [64]:
Paris_exhibitions.drop(Paris_exhibitions.loc[Paris_exhibitions['Mots clés'] == "A l'Aquarium tropical de Paris, le Spot est un mur numérique qui propose des rencontres inédites avec les baleines."].index, inplace=True)

In [65]:
Paris_exhibitions.head()

,URL_x,Titre,Chapeau,Description,Date de début,Date de fin,Mots clés,Nom du lieu,Adresse du lieu,Code postal,Ville,Coordonnées géographiques,Type de prix,audience,Année,REGION,PAYANT,GRATUIT,TOTAL,NOMDEP
0,https://www.paris.fr/evenements/baila-babibou-...,"Baila Babibou : spectacle danse & musique, dès...",Baila Babibou propose un dialogue entre une mé...,<p>La danseuse démarre le spectacle en chrysal...,2023-06-21T13:30:00+02:00,2023-06-21T14:00:00+02:00,"Danse,Enfants,Spectacle musical",Bibliothèque Jacqueline de Romilly,16 avenue de la Porte Montmartre,75018,Paris,"48.8994220173449, 2.33641997088512",gratuit,Public enfants. A partir de 1 ans. Jusqu'à 7 ans.,NaN,NaN,NaN,NaN,NaN,NaN
1,https://www.paris.fr/evenements/le-fil-des-noe...,Le fil des nœuds,Exposition collective de La.ima du 10 juin au ...,<p>Qu'y a-t-il au-delà de la peau qui cache la...,2023-06-10T02:00:00+02:00,2023-07-09T01:59:59+02:00,"Art contemporain,Expo,Photo",Médiathèque Edmond Rostand,11 rue Nicolas Chuquet,75017,Paris,"48.8883939892465, 2.30309000346023",gratuit,Public adolescents adultes.,NaN,NaN,NaN,NaN,NaN,NaN
2,https://www.paris.fr/evenements/parcours-d-int...,Parcours d’intellectuels en exil : un humanism...,Un cycle de 4 conférences dédié aux trajectoir...,<p>L’exil n’a jamais épargné les intellectuels...,2023-03-28T02:00:00+02:00,2023-06-14T01:59:59+02:00,"Conférence,Littérature,Sciences",Fondation Maison des sciences de l'homme,54 boulevard Raspail,75006,Paris,"48.8502890192206, 2.32696501646515",gratuit,Public adolescents adultes.,NaN,NaN,NaN,NaN,NaN,NaN
3,https://www.paris.fr/evenements/concert-java-m...,"Concert : Java, musique du pays Sunda",Venez découvrir la musique du pays Sunda de l'...,"<p>Basé à Paris, <strong>Gamelan Degung Sora G...",2023-05-26T22:00:00+02:00,2023-05-27T00:00:00+02:00,"Concert,Musique",Médiathèque Marguerite Yourcenar,"41, rue d'Alleray",75015,Paris,"48.83680697942, 2.30348101924335",gratuit,Tout public.,NaN,NaN,NaN,NaN,NaN,NaN
4,https://www.paris.fr/evenements/p-tit-dej-litt...,P'tit-Déj' Littéraire,Les bibliothécaires vous lisent leurs derniers...,"<p>Depuis quelques éditions, cette animation e...",2023-06-10T13:30:00+02:00,2023-06-10T15:00:00+02:00,Littérature,Médiathèque Marguerite Yourcenar,"41, rue d'Alleray",75015,Paris,"48.83680697942, 2.30348101924335",gratuit,Public adolescents adultes.,NaN,NaN,NaN,NaN,NaN,NaN


## Merging exhibitions & museums visitors 

In [66]:
visitors.head()

,REF DU MUSEE,NOM DU MUSEE,Année,REGION,VILLE,DATE APPELLATION,PAYANT,GRATUIT,TOTAL,Note,ID MUSEOFILE,NOMDEP,Observations
0,8109903,Musée de l'Abbaye Saint-Michel,2009,MIDI-PYRENEES,GAILLAC,01/02/2003,2613,669,3282,NaN,NaN,TARN,NaN
1,8114501,Musée Raymond Lafage,2009,MIDI-PYRENEES,LISLE-SUR-TARN,01/02/2003,1898,4429,6327,NaN,NaN,TARN,NaN
2,5904301,Musée Benoît de Puydt,2009,NORD-PAS-DE-CALAIS,BAILLEUL,01/02/2003,6590,0,6590,NaN,NaN,NORD,NaN
3,5912202,Musée Diocésain d'Art Sacré,2009,NORD-PAS-DE-CALAIS,CAMBRAI,01/02/2003,0,0,0,F,NaN,NORD,NaN
4,5918303,Musée Portuaire,2009,NORD-PAS-DE-CALAIS,DUNKERQUE,01/02/2003,24543,21642,46185,NaN,NaN,NORD,NaN


In [67]:
Paris_exhibitions.head()

,URL_x,Titre,Chapeau,Description,Date de début,Date de fin,Mots clés,Nom du lieu,Adresse du lieu,Code postal,Ville,Coordonnées géographiques,Type de prix,audience,Année,REGION,PAYANT,GRATUIT,TOTAL,NOMDEP
0,https://www.paris.fr/evenements/baila-babibou-...,"Baila Babibou : spectacle danse & musique, dès...",Baila Babibou propose un dialogue entre une mé...,<p>La danseuse démarre le spectacle en chrysal...,2023-06-21T13:30:00+02:00,2023-06-21T14:00:00+02:00,"Danse,Enfants,Spectacle musical",Bibliothèque Jacqueline de Romilly,16 avenue de la Porte Montmartre,75018,Paris,"48.8994220173449, 2.33641997088512",gratuit,Public enfants. A partir de 1 ans. Jusqu'à 7 ans.,NaN,NaN,NaN,NaN,NaN,NaN
1,https://www.paris.fr/evenements/le-fil-des-noe...,Le fil des nœuds,Exposition collective de La.ima du 10 juin au ...,<p>Qu'y a-t-il au-delà de la peau qui cache la...,2023-06-10T02:00:00+02:00,2023-07-09T01:59:59+02:00,"Art contemporain,Expo,Photo",Médiathèque Edmond Rostand,11 rue Nicolas Chuquet,75017,Paris,"48.8883939892465, 2.30309000346023",gratuit,Public adolescents adultes.,NaN,NaN,NaN,NaN,NaN,NaN
2,https://www.paris.fr/evenements/parcours-d-int...,Parcours d’intellectuels en exil : un humanism...,Un cycle de 4 conférences dédié aux trajectoir...,<p>L’exil n’a jamais épargné les intellectuels...,2023-03-28T02:00:00+02:00,2023-06-14T01:59:59+02:00,"Conférence,Littérature,Sciences",Fondation Maison des sciences de l'homme,54 boulevard Raspail,75006,Paris,"48.8502890192206, 2.32696501646515",gratuit,Public adolescents adultes.,NaN,NaN,NaN,NaN,NaN,NaN
3,https://www.paris.fr/evenements/concert-java-m...,"Concert : Java, musique du pays Sunda",Venez découvrir la musique du pays Sunda de l'...,"<p>Basé à Paris, <strong>Gamelan Degung Sora G...",2023-05-26T22:00:00+02:00,2023-05-27T00:00:00+02:00,"Concert,Musique",Médiathèque Marguerite Yourcenar,"41, rue d'Alleray",75015,Paris,"48.83680697942, 2.30348101924335",gratuit,Tout public.,NaN,NaN,NaN,NaN,NaN,NaN
4,https://www.paris.fr/evenements/p-tit-dej-litt...,P'tit-Déj' Littéraire,Les bibliothécaires vous lisent leurs derniers...,"<p>Depuis quelques éditions, cette animation e...",2023-06-10T13:30:00+02:00,2023-06-10T15:00:00+02:00,Littérature,Médiathèque Marguerite Yourcenar,"41, rue d'Alleray",75015,Paris,"48.83680697942, 2.30348101924335",gratuit,Public adolescents adultes.,NaN,NaN,NaN,NaN,NaN,NaN


In [68]:
df = exhibitions.merge(visitors, left_on="Nom du lieu", right_on="NOM DU MUSEE", how='left')
df.head()

,ID,URL,Titre,Chapeau,Description,Date de début,Date de fin,Occurrences,Description de la date,URL de l'image,...,REGION,VILLE,DATE APPELLATION,PAYANT,GRATUIT,TOTAL,Note,ID MUSEOFILE,NOMDEP,Observations
0,34155,https://www.paris.fr/evenements/baila-babibou-...,"Baila Babibou : spectacle danse & musique, dès...",Baila Babibou propose un dialogue entre une mé...,<p>La danseuse démarre le spectacle en chrysal...,2023-06-21T13:30:00+02:00,2023-06-21T14:00:00+02:00,2023-06-21T11:30:00+02:00_2023-06-21T12:00:00+...,Le mercredi 21 juin 2023<br />de 10h30 à 11h00...,https://cdn.paris.fr/qfapv4/2023/03/17/huge-06...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,34268,https://www.paris.fr/evenements/le-fil-des-noe...,Le fil des nœuds,Exposition collective de La.ima du 10 juin au ...,<p>Qu'y a-t-il au-delà de la peau qui cache la...,2023-06-10T02:00:00+02:00,2023-07-09T01:59:59+02:00,NaN,Du samedi 10 juin 2023 au samedi 08 juillet 20...,https://cdn.paris.fr/qfapv4/2023/03/19/huge-04...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,34245,https://www.paris.fr/evenements/parcours-d-int...,Parcours d’intellectuels en exil : un humanism...,Un cycle de 4 conférences dédié aux trajectoir...,<p>L’exil n’a jamais épargné les intellectuels...,2023-03-28T02:00:00+02:00,2023-06-14T01:59:59+02:00,NaN,Du mardi 28 mars 2023 au mardi 13 juin 2023 : ...,https://cdn.paris.fr/qfapv4/2023/03/18/huge-e3...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,33666,https://www.paris.fr/evenements/concert-java-m...,"Concert : Java, musique du pays Sunda",Venez découvrir la musique du pays Sunda de l'...,"<p>Basé à Paris, <strong>Gamelan Degung Sora G...",2023-05-26T22:00:00+02:00,2023-05-27T00:00:00+02:00,2023-05-26T20:00:00+02:00_2023-05-26T22:00:00+...,Le vendredi 26 mai 2023<br />de 19h00 à 21h00<...,https://cdn.paris.fr/qfapv4/2023/03/09/huge-7f...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,34251,https://www.paris.fr/evenements/p-tit-dej-litt...,P'tit-Déj' Littéraire,Les bibliothécaires vous lisent leurs derniers...,"<p>Depuis quelques éditions, cette animation e...",2023-06-10T13:30:00+02:00,2023-06-10T15:00:00+02:00,2023-06-10T11:30:00+02:00_2023-06-10T13:00:00+...,Le samedi 10 juin 2023<br />de 10h30 à 12h00<b...,https://cdn.paris.fr/qfapv4/2023/03/04/huge-a3...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [69]:
df.drop(columns=['ID', "Texte alternatif de l'image", 'Accès PMR', 'Accès mal voyant', 'Accès mal entendant', 'Transport', 'Url de contact', 'Téléphone de contact', 'Email de contact', 'En ligne - address_url', 'En ligne - address_url_text', 'En ligne - address_text', 'title_event', 'childrens', 'contributor_group', 'Programmes', 'Détail du prix', 'Occurrences', 'Observations'], inplace=True)

In [70]:
len(df)

2653

In [71]:
Paris_exhibitions_visitors = df.loc[df["NOM DU MUSEE"].notna()]

In [72]:
len(Paris_exhibitions_visitors)

613

In [73]:
#613 museums where we can retrieve infos about the visitors (e.g. PAYANT / GRATUIT / etc)

In [74]:
Paris_exhibitions_visitors.head()

,URL,Titre,Chapeau,Description,Date de début,Date de fin,Description de la date,URL de l'image,Crédit de l'image,Mots clés,...,Année,REGION,VILLE,DATE APPELLATION,PAYANT,GRATUIT,TOTAL,Note,ID MUSEOFILE,NOMDEP
26,https://www.paris.fr/evenements/nouveaute-baby...,Nouveauté ! Baby-visite au musée Bourdelle,Profitez d'une visite sensorielle et d'un temp...,<p>Une \nvisite sensorielle et participative a...,2023-05-10T13:30:00+02:00,2023-06-14T14:30:00+02:00,Le mercredi 14 juin 2023<br />de 10h30 à 11h30...,https://cdn.paris.fr/qfapv4/2023/04/26/huge-00...,(c) ELODIE RATSIMBAZAFY Paris Musées,"Atelier,Enfants",...,2011.0,ILE-DE-FRANCE,PARIS,08/07/2004,81804.0,40720.0,122524.0,NaN,NaN,PARIS
27,https://www.paris.fr/evenements/nouveaute-baby...,Nouveauté ! Baby-visite au musée Bourdelle,Profitez d'une visite sensorielle et d'un temp...,<p>Une \nvisite sensorielle et participative a...,2023-05-10T13:30:00+02:00,2023-06-14T14:30:00+02:00,Le mercredi 14 juin 2023<br />de 10h30 à 11h30...,https://cdn.paris.fr/qfapv4/2023/04/26/huge-00...,(c) ELODIE RATSIMBAZAFY Paris Musées,"Atelier,Enfants",...,2013.0,ILE-DE-FRANCE,PARIS,08/07/2004,12433.0,54170.0,66603.0,NaN,NaN,PARIS
28,https://www.paris.fr/evenements/nouveaute-baby...,Nouveauté ! Baby-visite au musée Bourdelle,Profitez d'une visite sensorielle et d'un temp...,<p>Une \nvisite sensorielle et participative a...,2023-05-10T13:30:00+02:00,2023-06-14T14:30:00+02:00,Le mercredi 14 juin 2023<br />de 10h30 à 11h30...,https://cdn.paris.fr/qfapv4/2023/04/26/huge-00...,(c) ELODIE RATSIMBAZAFY Paris Musées,"Atelier,Enfants",...,2016.0,ILE-DE-FRANCE,PARIS,08/07/2004,2720.0,36657.0,39377.0,NaN,NaN,PARIS
29,https://www.paris.fr/evenements/nouveaute-baby...,Nouveauté ! Baby-visite au musée Bourdelle,Profitez d'une visite sensorielle et d'un temp...,<p>Une \nvisite sensorielle et participative a...,2023-05-10T13:30:00+02:00,2023-06-14T14:30:00+02:00,Le mercredi 14 juin 2023<br />de 10h30 à 11h30...,https://cdn.paris.fr/qfapv4/2023/04/26/huge-00...,(c) ELODIE RATSIMBAZAFY Paris Musées,"Atelier,Enfants",...,2005.0,ILE-DE-FRANCE,PARIS,08/07/2004,9840.0,28912.0,38752.0,NaN,NaN,PARIS
30,https://www.paris.fr/evenements/nouveaute-baby...,Nouveauté ! Baby-visite au musée Bourdelle,Profitez d'une visite sensorielle et d'un temp...,<p>Une \nvisite sensorielle et participative a...,2023-05-10T13:30:00+02:00,2023-06-14T14:30:00+02:00,Le mercredi 14 juin 2023<br />de 10h30 à 11h30...,https://cdn.paris.fr/qfapv4/2023/04/26/huge-00...,(c) ELODIE RATSIMBAZAFY Paris Musées,"Atelier,Enfants",...,2018.0,ILE-DE-FRANCE,PARIS,08/07/2004,18500.0,39322.0,57822.0,NaN,NaN,PARIS


In [75]:
Paris_exhibitions_visitors.isna().sum()

URL                             0
Titre                           0
Chapeau                         0
Description                     0
Date de début                   0
Date de fin                     0
Description de la date          0
URL de l'image                  0
Crédit de l'image               0
Mots clés                       0
Nom du lieu                     0
Adresse du lieu                 0
Code postal                     0
Ville                           0
Coordonnées géographiques       0
URL Facebook associée         291
URL Twitter associée          588
Type de prix                    0
Type d'accès                    0
URL de réservation            246
URL de réservation - Texte    308
Date de mise à jour             0
Image de couverture             0
audience                        0
REF DU MUSEE                    0
NOM DU MUSEE                    0
Année                           0
REGION                          0
VILLE                           0
DATE APPELLATI

In [76]:
#Should remove the ID, the URL, the image URL, the Texte alternatif de l'image, the Crédit de l'image, the Accès PMR, the Accès mal voyant, the Accès mal entendant, the Transport (only null values), the URL de contact, the Téléphone de contact, the Email de contact, the Programmes, the En ligne - address_url, the En ligne - address_url_text, the En ligne - address_text
#The title_event (duplicate of the Titre)
#The childrens
#The DATE APPELLATION
#The Note
#The ID MUSEOFILE -> TBC
#The Observations

In [77]:
Paris_exhibitions_visitors.head()

,URL,Titre,Chapeau,Description,Date de début,Date de fin,Description de la date,URL de l'image,Crédit de l'image,Mots clés,...,Année,REGION,VILLE,DATE APPELLATION,PAYANT,GRATUIT,TOTAL,Note,ID MUSEOFILE,NOMDEP
26,https://www.paris.fr/evenements/nouveaute-baby...,Nouveauté ! Baby-visite au musée Bourdelle,Profitez d'une visite sensorielle et d'un temp...,<p>Une \nvisite sensorielle et participative a...,2023-05-10T13:30:00+02:00,2023-06-14T14:30:00+02:00,Le mercredi 14 juin 2023<br />de 10h30 à 11h30...,https://cdn.paris.fr/qfapv4/2023/04/26/huge-00...,(c) ELODIE RATSIMBAZAFY Paris Musées,"Atelier,Enfants",...,2011.0,ILE-DE-FRANCE,PARIS,08/07/2004,81804.0,40720.0,122524.0,NaN,NaN,PARIS
27,https://www.paris.fr/evenements/nouveaute-baby...,Nouveauté ! Baby-visite au musée Bourdelle,Profitez d'une visite sensorielle et d'un temp...,<p>Une \nvisite sensorielle et participative a...,2023-05-10T13:30:00+02:00,2023-06-14T14:30:00+02:00,Le mercredi 14 juin 2023<br />de 10h30 à 11h30...,https://cdn.paris.fr/qfapv4/2023/04/26/huge-00...,(c) ELODIE RATSIMBAZAFY Paris Musées,"Atelier,Enfants",...,2013.0,ILE-DE-FRANCE,PARIS,08/07/2004,12433.0,54170.0,66603.0,NaN,NaN,PARIS
28,https://www.paris.fr/evenements/nouveaute-baby...,Nouveauté ! Baby-visite au musée Bourdelle,Profitez d'une visite sensorielle et d'un temp...,<p>Une \nvisite sensorielle et participative a...,2023-05-10T13:30:00+02:00,2023-06-14T14:30:00+02:00,Le mercredi 14 juin 2023<br />de 10h30 à 11h30...,https://cdn.paris.fr/qfapv4/2023/04/26/huge-00...,(c) ELODIE RATSIMBAZAFY Paris Musées,"Atelier,Enfants",...,2016.0,ILE-DE-FRANCE,PARIS,08/07/2004,2720.0,36657.0,39377.0,NaN,NaN,PARIS
29,https://www.paris.fr/evenements/nouveaute-baby...,Nouveauté ! Baby-visite au musée Bourdelle,Profitez d'une visite sensorielle et d'un temp...,<p>Une \nvisite sensorielle et participative a...,2023-05-10T13:30:00+02:00,2023-06-14T14:30:00+02:00,Le mercredi 14 juin 2023<br />de 10h30 à 11h30...,https://cdn.paris.fr/qfapv4/2023/04/26/huge-00...,(c) ELODIE RATSIMBAZAFY Paris Musées,"Atelier,Enfants",...,2005.0,ILE-DE-FRANCE,PARIS,08/07/2004,9840.0,28912.0,38752.0,NaN,NaN,PARIS
30,https://www.paris.fr/evenements/nouveaute-baby...,Nouveauté ! Baby-visite au musée Bourdelle,Profitez d'une visite sensorielle et d'un temp...,<p>Une \nvisite sensorielle et participative a...,2023-05-10T13:30:00+02:00,2023-06-14T14:30:00+02:00,Le mercredi 14 juin 2023<br />de 10h30 à 11h30...,https://cdn.paris.fr/qfapv4/2023/04/26/huge-00...,(c) ELODIE RATSIMBAZAFY Paris Musées,"Atelier,Enfants",...,2018.0,ILE-DE-FRANCE,PARIS,08/07/2004,18500.0,39322.0,57822.0,NaN,NaN,PARIS


In [78]:
Paris_exhibitions_visitors.drop(columns=['Note', 'URL Facebook associée', 'URL Twitter associée', 'URL de réservation', 'URL de réservation - Texte', "URL de l'image", "Crédit de l'image", "Description de la date", "Type d'accès", "Date de mise à jour", "Image de couverture", "REF DU MUSEE", "VILLE", "DATE APPELLATION", "NOM DU MUSEE", "ID MUSEOFILE"], inplace=True)

/var/folders/xp/jp6pr1sn3p50vy46h4q0ntqr0000gn/T/ipykernel_60167/2752707509.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Paris_exhibitions_visitors.drop(columns=['Note', 'URL Facebook associée', 'URL Twitter associée', 'URL de réservation', 'URL de réservation - Texte', "URL de l'image", "Crédit de l'image", "Description de la date", "Type d'accès", "Date de mise à jour", "Image de couverture", "REF DU MUSEE", "VILLE", "DATE APPELLATION", "NOM DU MUSEE", "ID MUSEOFILE"], inplace=True)


In [79]:
len(Paris_exhibitions_visitors)

613

In [80]:
Paris_exhibitions_visitors.isna().sum() #OK

URL                          0
Titre                        0
Chapeau                      0
Description                  0
Date de début                0
Date de fin                  0
Mots clés                    0
Nom du lieu                  0
Adresse du lieu              0
Code postal                  0
Ville                        0
Coordonnées géographiques    0
Type de prix                 0
audience                     0
Année                        0
REGION                       0
PAYANT                       0
GRATUIT                      0
TOTAL                        0
NOMDEP                       0
dtype: int64

In [81]:
len(Paris_exhibitions)

2047

In [82]:
Paris_exhibitions.isna().sum() #for the NLP, should remove "REGION", "PAYANT", "GRATUIT", "TOTAL", "NOMDEP", "ANNEE"

URL_x                           0
Titre                           0
Chapeau                         2
Description                    19
Date de début                  31
Date de fin                    31
Mots clés                       0
Nom du lieu                   119
Adresse du lieu               116
Code postal                   116
Ville                         116
Coordonnées géographiques      71
Type de prix                    4
audience                        0
Année                        1988
REGION                       1988
PAYANT                       1988
GRATUIT                      1988
TOTAL                        1988
NOMDEP                       1988
dtype: int64